In [1]:
import random
from multiprocess import Pool
import numpy as np
import itertools
from typing import Callable

In [2]:
class Ant:
    def __init__(self, max_lifespan: int =float('inf')): #int type does not define inf.
        """
        Ant object.
        :param max_lifespan: In case of an spatially unbounded problem this variable defines how long the ant survives until the simulation stops.
        """
        self.x_position: int = 0 #current x, y positions
        self.y_position: int = 0
        self.food: bool = False #found food?
        self.num_movements: int = 0 #time passed in seconds

        #define max runtime for unbounded problems
        self.is_alive: bool = True
        self.max_lifespan: int = max_lifespan

    def move(self, movement: list) -> None:
        """
        Moves the ant by x, y;
        If the current position is [10,0] and you pass in [0,10], the new position will be [10, 10]
        :param movement: x, y change
        :return: None
        """
        self.x_position += movement[0]
        self.y_position += movement[1]
        self.num_movements += 1

    def found_food(self, food_function: Callable[[int, int], bool]) -> None:
        """
        Checks if the ant reached a point with food.
        :param food_function: A callable object that receives the current x, y coordinates and returns a bool indicating if food is present.
        :return: None
        """
        self.food = food_function(self.x_position, self.y_position)

        if (self.num_movements > self.max_lifespan) and not self.food:
            self.is_alive = False



In [3]:
direction_movement = {"north":[0,10], "south": [0,-10], "east":[10,0], "west":[-10,0]} #x and y shifts for each movement type
directions = list(direction_movement.keys())

def simulate(ant: Ant, food_func: Callable[[int, int], bool]) -> list:
    """
    Simulates the random walk of an ant.
    :param ant: requires an Ant object.
    :param food_func: A callable object that receives the current x, y coordinates and returns a bool.
    :return: A list containing if the ant is alive and the number of movements it took until its found food. If its dead the number of movements equals the maximum lifespan of an ant.
    """
    while not ant.food and ant.is_alive:
        movement = direction_movement[random.choice(directions)] #picks a random direction for the ant
        ant.move(movement)
        ant.found_food(food_func)
    return [ant.is_alive, ant.num_movements]


# Question 1
If the food is located on east-west lines 20cm to the north and 20cm to the south,
as well as on north-south lines 20cm to the east and 20cm to the west from the anthill,
how long will it take the ant to reach it on average?

In [8]:
num_ants = 1000000

# food bound for Q1
def square_food(x: int, y: int) -> bool:
    return (abs(x) == 20 or abs(y) == 20)

ants = [Ant() for i in range(num_ants)]
p = Pool(8)
res = p.starmap(simulate, zip(ants, itertools.repeat(square_food))) #starmap = multi-argument version of map

# getting all alive ants; important for unbounded problems
results = [r[1] for r in res if r[0]]

print("Average time taken: ", np.sum(results)/len(results))
print("% alive ants: ", len(results)/num_ants)

Average time taken:  4.497802
% alive ants:  1.0


# Question 2
What is the average time the ant will reach food if it is located only
on a diagonal line passing through (10cm, 0cm) and (0cm, 10cm) points?

In [5]:
num_ants = 1000000
unbounded_food_line = lambda x, y: -x + 10 == y #if ant is on diagonal line:     y = -mx + c

ants = [Ant(max_lifespan=100) for i in range(num_ants)]
p = Pool(8)
res = p.starmap(simulate, zip(ants, itertools.repeat(unbounded_food_line)))

results = [r[1] for r in res if r[0]]

print("Average time to food for all alive ants (max_lifespan=100): ", np.sum(results) / len(results))
print("All ants who didnt find food are assumed to be dead.")
print("% alive ants: ", len(results)/num_ants)
print("It's an unbounded problem with an increasing time limit the average time increases and approaches infinity.")

Average time to food for all alive ants (max_lifespan=100):  7.725524680501818
All ants who didnt find food are assumed to be dead.
% alive ants:  0.921132
It's an unbounded problem with an increasing time limit the average time increases and approaches infinity.


# Question 3
Can you write a program that comes up with an estimate of average time to find food for any
closed boundary around the anthill?
What would be the answer if food is located outside an defined by
((x – 2.5cm) / 30cm )2 + ( (y – 2.5cm) / 40cm )2 < 1 in coordinate system
where the anthill is located at (x = 0cm, y = 0cm)?
Provide us with a solution rounded to the nearest integer.

In [10]:
num_ants = 1000000
bounded_area = lambda x, y: ((x-2.5)/30)**2 + ((y-2.5)/40)**2 >= 1 #returns true as soon as ant leaves the area defined by the function (i.e. found food)

ants = [Ant() for i in range(num_ants)]
p = Pool(8)
res = p.starmap(simulate, zip(ants, itertools.repeat(bounded_area)))

results = [r[1] for r in res if r[0]]

print("Average time taken: ", round(np.sum(results) / len(results)))
print("% alive ants: ", len(results)/num_ants)

Average time taken:  14
% alive ants:  1.0
